# 🌍 AI-Powered Trip Planner - Multi-Agent System

This notebook implements a sophisticated multi-agent system for trip planning using:
- **LangGraph** for orchestration
- **Gemini 2.5 Flash** as the LLM
- **Multiple Specialized Agents**: Planner, Researcher, Budget Checker
- **Custom Tools**: Search, Distance Calculator, Budget Calculator, Local Catalog

## Architecture Overview

```
User Input → Planner Agent → Researcher Agent → Budget Agent → Final Itinerary
              ↓                ↓                  ↓
           Tools            Tools              Tools
```

## 📦 Installation & Setup

In [ ]:
# Install required packages
!pip install -q langgraph langchain-google-genai langchain-core langchain-community python-dotenv

In [ ]:
# Import required libraries
import os
import json
from datetime import datetime, timedelta
from typing import TypedDict, Annotated, List, Dict, Any
from operator import add

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Set up API key
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Initialize Gemini 2.5 Flash
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    max_tokens=2048
)

print("✅ Setup complete!")

## 🗄️ Local Destination Catalog

We'll create a local database of destinations with attractions, costs, and details.

In [ ]:
# Create local destination catalog
DESTINATION_CATALOG = {
    "paris": {
        "name": "Paris, France",
        "attractions": [
            {"name": "Eiffel Tower", "category": "landmark", "duration_hours": 2, "cost_usd": 25, "rating": 4.7},
            {"name": "Louvre Museum", "category": "museum", "duration_hours": 4, "cost_usd": 20, "rating": 4.8},
            {"name": "Notre-Dame Cathedral", "category": "landmark", "duration_hours": 1.5, "cost_usd": 0, "rating": 4.6},
            {"name": "Arc de Triomphe", "category": "landmark", "duration_hours": 1, "cost_usd": 13, "rating": 4.5},
            {"name": "Sacré-Cœur", "category": "landmark", "duration_hours": 1.5, "cost_usd": 0, "rating": 4.7},
            {"name": "Versailles Palace", "category": "museum", "duration_hours": 5, "cost_usd": 30, "rating": 4.8},
            {"name": "Seine River Cruise", "category": "activity", "duration_hours": 1.5, "cost_usd": 15, "rating": 4.4},
            {"name": "Montmartre Walking Tour", "category": "activity", "duration_hours": 2, "cost_usd": 0, "rating": 4.6}
        ],
        "avg_meal_cost": {"budget": 15, "mid-range": 35, "luxury": 80},
        "avg_hotel_cost": {"budget": 60, "mid-range": 150, "luxury": 350},
        "local_transport_day_pass": 15
    },
    "tokyo": {
        "name": "Tokyo, Japan",
        "attractions": [
            {"name": "Senso-ji Temple", "category": "landmark", "duration_hours": 2, "cost_usd": 0, "rating": 4.6},
            {"name": "Tokyo Skytree", "category": "landmark", "duration_hours": 2, "cost_usd": 25, "rating": 4.5},
            {"name": "Meiji Shrine", "category": "landmark", "duration_hours": 1.5, "cost_usd": 0, "rating": 4.7},
            {"name": "Tsukiji Fish Market", "category": "activity", "duration_hours": 2, "cost_usd": 0, "rating": 4.5},
            {"name": "Tokyo National Museum", "category": "museum", "duration_hours": 3, "cost_usd": 7, "rating": 4.6},
            {"name": "Shibuya Crossing", "category": "activity", "duration_hours": 1, "cost_usd": 0, "rating": 4.4},
            {"name": "Akihabara District", "category": "activity", "duration_hours": 3, "cost_usd": 0, "rating": 4.5},
            {"name": "Imperial Palace", "category": "landmark", "duration_hours": 2, "cost_usd": 0, "rating": 4.4}
        ],
        "avg_meal_cost": {"budget": 10, "mid-range": 25, "luxury": 70},
        "avg_hotel_cost": {"budget": 50, "mid-range": 120, "luxury": 300},
        "local_transport_day_pass": 10
    },
    "new york": {
        "name": "New York City, USA",
        "attractions": [
            {"name": "Statue of Liberty", "category": "landmark", "duration_hours": 3, "cost_usd": 25, "rating": 4.7},
            {"name": "Central Park", "category": "activity", "duration_hours": 3, "cost_usd": 0, "rating": 4.8},
            {"name": "Metropolitan Museum of Art", "category": "museum", "duration_hours": 4, "cost_usd": 30, "rating": 4.8},
            {"name": "Times Square", "category": "activity", "duration_hours": 1.5, "cost_usd": 0, "rating": 4.4},
            {"name": "Empire State Building", "category": "landmark", "duration_hours": 2, "cost_usd": 44, "rating": 4.6},
            {"name": "Brooklyn Bridge Walk", "category": "activity", "duration_hours": 1.5, "cost_usd": 0, "rating": 4.7},
            {"name": "9/11 Memorial", "category": "landmark", "duration_hours": 2, "cost_usd": 0, "rating": 4.8},
            {"name": "Broadway Show", "category": "activity", "duration_hours": 3, "cost_usd": 150, "rating": 4.9}
        ],
        "avg_meal_cost": {"budget": 15, "mid-range": 40, "luxury": 100},
        "avg_hotel_cost": {"budget": 100, "mid-range": 250, "luxury": 500},
        "local_transport_day_pass": 13
    },
    "bali": {
        "name": "Bali, Indonesia",
        "attractions": [
            {"name": "Tanah Lot Temple", "category": "landmark", "duration_hours": 2, "cost_usd": 5, "rating": 4.5},
            {"name": "Ubud Monkey Forest", "category": "activity", "duration_hours": 2, "cost_usd": 7, "rating": 4.4},
            {"name": "Tegallalang Rice Terraces", "category": "activity", "duration_hours": 2, "cost_usd": 2, "rating": 4.6},
            {"name": "Uluwatu Temple", "category": "landmark", "duration_hours": 2, "cost_usd": 5, "rating": 4.7},
            {"name": "Water Temple Tour", "category": "activity", "duration_hours": 3, "cost_usd": 15, "rating": 4.6},
            {"name": "Beach Day - Seminyak", "category": "activity", "duration_hours": 4, "cost_usd": 0, "rating": 4.5},
            {"name": "Balinese Cooking Class", "category": "activity", "duration_hours": 4, "cost_usd": 35, "rating": 4.8},
            {"name": "Mount Batur Sunrise Trek", "category": "activity", "duration_hours": 6, "cost_usd": 40, "rating": 4.9}
        ],
        "avg_meal_cost": {"budget": 5, "mid-range": 15, "luxury": 40},
        "avg_hotel_cost": {"budget": 25, "mid-range": 70, "luxury": 200},
        "local_transport_day_pass": 5
    }
}

# Save to JSON file
with open('destination_catalog.json', 'w') as f:
    json.dump(DESTINATION_CATALOG, f, indent=2)

print("✅ Destination catalog created with", len(DESTINATION_CATALOG), "destinations")

## 🛠️ Custom Tools

We'll create specialized tools for our agents to use.

In [ ]:
@tool
def search_destination_catalog(destination: str, interest_category: str = None) -> str:
    """
    Search the local destination catalog for attractions and information.
    
    Args:
        destination: The destination city (e.g., 'paris', 'tokyo', 'new york', 'bali')
        interest_category: Optional filter by category ('landmark', 'museum', 'activity')
    
    Returns:
        JSON string with destination information and attractions
    """
    destination_key = destination.lower().strip()
    
    if destination_key not in DESTINATION_CATALOG:
        return json.dumps({"error": f"Destination '{destination}' not found in catalog. Available: {list(DESTINATION_CATALOG.keys())}"})
    
    dest_data = DESTINATION_CATALOG[destination_key].copy()
    
    # Filter by category if specified
    if interest_category:
        dest_data['attractions'] = [
            attr for attr in dest_data['attractions'] 
            if attr['category'].lower() == interest_category.lower()
        ]
    
    return json.dumps(dest_data, indent=2)


@tool
def calculate_daily_budget(destination: str, travel_style: str, num_activities: int = 3) -> str:
    """
    Calculate estimated daily budget for a destination.
    
    Args:
        destination: The destination city
        travel_style: 'budget', 'mid-range', or 'luxury'
        num_activities: Number of paid activities per day (default: 3)
    
    Returns:
        JSON string with budget breakdown
    """
    destination_key = destination.lower().strip()
    
    if destination_key not in DESTINATION_CATALOG:
        return json.dumps({"error": f"Destination '{destination}' not found"})
    
    dest_data = DESTINATION_CATALOG[destination_key]
    style = travel_style.lower()
    
    if style not in ['budget', 'mid-range', 'luxury']:
        style = 'mid-range'
    
    # Calculate costs
    hotel_cost = dest_data['avg_hotel_cost'][style]
    meal_cost = dest_data['avg_meal_cost'][style] * 3  # 3 meals
    transport_cost = dest_data['local_transport_day_pass']
    
    # Average activity cost
    avg_activity_cost = sum(a['cost_usd'] for a in dest_data['attractions']) / len(dest_data['attractions'])
    activity_cost = avg_activity_cost * num_activities
    
    total_daily = hotel_cost + meal_cost + transport_cost + activity_cost
    
    budget_breakdown = {
        "destination": dest_data['name'],
        "travel_style": style,
        "daily_breakdown": {
            "accommodation": round(hotel_cost, 2),
            "meals": round(meal_cost, 2),
            "local_transport": round(transport_cost, 2),
            "activities": round(activity_cost, 2)
        },
        "total_daily_cost_usd": round(total_daily, 2),
        "currency": "USD"
    }
    
    return json.dumps(budget_breakdown, indent=2)


@tool
def calculate_trip_duration(num_days: int, activities_per_day: int) -> str:
    """
    Calculate optimal time allocation for activities.
    
    Args:
        num_days: Number of days for the trip
        activities_per_day: Target number of activities per day
    
    Returns:
        JSON string with time allocation recommendations
    """
    # Assume 8-10 hours of active time per day
    active_hours_per_day = 9
    
    # Account for meals (1.5 hours each for 3 meals)
    meal_time = 4.5
    
    # Available time for activities
    available_hours = active_hours_per_day - meal_time
    
    # Time per activity
    time_per_activity = available_hours / activities_per_day if activities_per_day > 0 else 0
    
    result = {
        "trip_duration_days": num_days,
        "active_hours_per_day": active_hours_per_day,
        "meal_time_hours": meal_time,
        "available_activity_hours": round(available_hours, 1),
        "activities_per_day": activities_per_day,
        "avg_time_per_activity_hours": round(time_per_activity, 1),
        "total_activities": num_days * activities_per_day,
        "recommendation": f"Plan {activities_per_day} activities per day, each lasting approximately {round(time_per_activity, 1)} hours"
    }
    
    return json.dumps(result, indent=2)


@tool
def get_activity_recommendations(destination: str, interests: List[str], max_results: int = 5) -> str:
    """
    Get personalized activity recommendations based on interests.
    
    Args:
        destination: The destination city
        interests: List of interest categories ('landmark', 'museum', 'activity')
        max_results: Maximum number of recommendations to return
    
    Returns:
        JSON string with recommended activities
    """
    destination_key = destination.lower().strip()
    
    if destination_key not in DESTINATION_CATALOG:
        return json.dumps({"error": f"Destination '{destination}' not found"})
    
    dest_data = DESTINATION_CATALOG[destination_key]
    attractions = dest_data['attractions']
    
    # Filter by interests
    if interests:
        filtered = [a for a in attractions if a['category'] in interests]
    else:
        filtered = attractions
    
    # Sort by rating
    sorted_attractions = sorted(filtered, key=lambda x: x['rating'], reverse=True)
    
    # Limit results
    recommendations = sorted_attractions[:max_results]
    
    result = {
        "destination": dest_data['name'],
        "interests_matched": interests if interests else "all",
        "recommendations": recommendations,
        "total_found": len(recommendations)
    }
    
    return json.dumps(result, indent=2)


# Collect all tools
tools = [
    search_destination_catalog,
    calculate_daily_budget,
    calculate_trip_duration,
    get_activity_recommendations
]

print(f"✅ Created {len(tools)} custom tools")

## 🤖 Agent State Definition

Define the shared state that flows through our agent graph.

In [ ]:
class TripPlannerState(TypedDict):
    """State shared across all agents in the trip planning workflow."""
    
    # User inputs
    user_request: str
    destination: str
    budget_usd: float
    num_days: int
    travel_style: str  # budget, mid-range, luxury
    interests: List[str]  # landmark, museum, activity
    
    # Agent outputs
    messages: Annotated[List, add]  # Conversation history
    research_data: Dict[str, Any]  # Data from researcher agent
    budget_analysis: Dict[str, Any]  # Data from budget agent
    itinerary: Dict[str, Any]  # Final itinerary from planner
    
    # Control flow
    next_agent: str  # Which agent to call next
    iteration_count: int  # Track iterations

print("✅ State definition created")

## 🎯 Specialized Agents

We'll create three specialized agents:
1. **Researcher Agent**: Gathers destination information
2. **Budget Agent**: Analyzes costs and validates budget
3. **Planner Agent**: Creates the final itinerary

In [ ]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

def researcher_agent(state: TripPlannerState) -> TripPlannerState:
    """
    Researcher Agent: Gathers information about the destination.
    Uses tools to search catalog and get activity recommendations.
    """
    print("\n🔍 Researcher Agent activated...")
    
    system_message = SystemMessage(content="""You are a Destination Research Agent.
    Your role is to gather comprehensive information about travel destinations.
    
    Use the available tools to:
    1. Search the destination catalog for general information
    2. Get activity recommendations based on user interests
    3. Gather details about attractions, costs, and durations
    
    Provide detailed, factual information to help plan the trip.
    """)
    
    user_message = HumanMessage(content=f"""Research the destination: {state['destination']}
    
    User interests: {', '.join(state['interests'])}
    Trip duration: {state['num_days']} days
    Travel style: {state['travel_style']}
    
    Please gather:
    1. General destination information
    2. Top recommended activities matching user interests
    3. Typical costs and durations for activities
    """)
    
    messages = [system_message, user_message] + state.get('messages', [])
    
    # Agent reasoning loop
    research_results = {}
    max_iterations = 3
    
    for i in range(max_iterations):
        response = llm_with_tools.invoke(messages)
        messages.append(response)
        
        # Check if agent wants to use tools
        if response.tool_calls:
            print(f"  📞 Calling {len(response.tool_calls)} tool(s)...")
            
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                # Execute tool
                tool_func = next(t for t in tools if t.name == tool_name)
                tool_result = tool_func.invoke(tool_args)
                
                print(f"    ✓ {tool_name}: {tool_args}")
                
                # Add tool result to messages
                messages.append(AIMessage(content=f"Tool {tool_name} result: {tool_result}"))
                
                # Store in research results
                research_results[tool_name] = json.loads(tool_result) if isinstance(tool_result, str) else tool_result
        else:
            # No more tools to call, agent is done
            break
    
    # Final summary from researcher
    summary_prompt = HumanMessage(content="Based on your research, provide a concise summary of the destination and top recommendations.")
    messages.append(summary_prompt)
    final_response = llm.invoke(messages)
    
    print("  ✅ Research complete")
    
    return {
        "messages": [final_response],
        "research_data": research_results,
        "next_agent": "budget_agent"
    }


def budget_agent(state: TripPlannerState) -> TripPlannerState:
    """
    Budget Agent: Analyzes costs and validates against user budget.
    Uses budget calculation tools to estimate trip costs.
    """
    print("\n💰 Budget Agent activated...")
    
    system_message = SystemMessage(content="""You are a Budget Analysis Agent.
    Your role is to analyze trip costs and ensure they fit within the user's budget.
    
    Use the available tools to:
    1. Calculate daily budget requirements
    2. Estimate total trip costs
    3. Provide budget recommendations and adjustments if needed
    
    Be realistic and transparent about costs.
    """)
    
    user_message = HumanMessage(content=f"""Analyze the budget for this trip:
    
    Destination: {state['destination']}
    Duration: {state['num_days']} days
    Travel style: {state['travel_style']}
    User budget: ${state['budget_usd']} USD
    
    Research data available: {json.dumps(state.get('research_data', {}), indent=2)}
    
    Please:
    1. Calculate estimated daily costs
    2. Calculate total trip cost
    3. Compare with user budget
    4. Provide recommendations if budget adjustments are needed
    """)
    
    messages = [system_message, user_message]
    
    # Agent reasoning loop
    budget_results = {}
    max_iterations = 3
    
    for i in range(max_iterations):
        response = llm_with_tools.invoke(messages)
        messages.append(response)
        
        if response.tool_calls:
            print(f"  📞 Calling {len(response.tool_calls)} tool(s)...")
            
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                tool_func = next(t for t in tools if t.name == tool_name)
                tool_result = tool_func.invoke(tool_args)
                
                print(f"    ✓ {tool_name}: {tool_args}")
                
                messages.append(AIMessage(content=f"Tool {tool_name} result: {tool_result}"))
                budget_results[tool_name] = json.loads(tool_result) if isinstance(tool_result, str) else tool_result
        else:
            break
    
    # Final budget analysis
    summary_prompt = HumanMessage(content="Provide a final budget analysis with recommendations.")
    messages.append(summary_prompt)
    final_response = llm.invoke(messages)
    
    print("  ✅ Budget analysis complete")
    
    return {
        "messages": [final_response],
        "budget_analysis": budget_results,
        "next_agent": "planner_agent"
    }


def planner_agent(state: TripPlannerState) -> TripPlannerState:
    """
    Planner Agent: Creates the final day-by-day itinerary.
    Synthesizes information from researcher and budget agents.
    """
    print("\n📋 Planner Agent activated...")
    
    system_message = SystemMessage(content="""You are a Trip Planner Agent.
    Your role is to create detailed, personalized day-by-day itineraries.
    
    Based on research and budget data, create an itinerary that:
    1. Fits within the user's budget
    2. Matches their interests and travel style
    3. Has realistic timing and pacing
    4. Includes cost estimates for each activity
    5. Explains why activities were chosen
    
    Format the itinerary clearly with day-by-day breakdown.
    """)
    
    user_message = HumanMessage(content=f"""Create a detailed itinerary for:
    
    Destination: {state['destination']}
    Duration: {state['num_days']} days
    Budget: ${state['budget_usd']} USD
    Travel style: {state['travel_style']}
    Interests: {', '.join(state['interests'])}
    
    Research findings:
    {json.dumps(state.get('research_data', {}), indent=2)}
    
    Budget analysis:
    {json.dumps(state.get('budget_analysis', {}), indent=2)}
    
    Create a day-by-day itinerary with:
    - Morning, afternoon, and evening activities
    - Approximate timings
    - Cost estimates for each activity
    - Brief explanation of why each activity was chosen
    - Daily cost summary
    - Total trip cost
    """)
    
    messages = [system_message, user_message]
    
    # Generate itinerary
    response = llm.invoke(messages)
    
    print("  ✅ Itinerary created")
    
    # Structure the itinerary
    itinerary = {
        "destination": state['destination'],
        "duration_days": state['num_days'],
        "budget_usd": state['budget_usd'],
        "travel_style": state['travel_style'],
        "itinerary_text": response.content,
        "created_at": datetime.now().isoformat()
    }
    
    return {
        "messages": [response],
        "itinerary": itinerary,
        "next_agent": "END"
    }

print("✅ All agents defined")

## 🔄 LangGraph Orchestration

Build the agent workflow graph using LangGraph.

In [ ]:
# Create the graph
workflow = StateGraph(TripPlannerState)

# Add agent nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("budget_checker", budget_agent)
workflow.add_node("planner", planner_agent)

# Define the flow
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "budget_checker")
workflow.add_edge("budget_checker", "planner")
workflow.add_edge("planner", END)

# Compile the graph
app = workflow.compile()

print("✅ LangGraph workflow compiled")
print("\n📊 Workflow: Researcher → Budget Checker → Planner → END")

## 🎬 Run the Trip Planner

Let's test our multi-agent system with a sample trip request!

In [ ]:
# Sample user request
user_input = {
    "user_request": "Plan a trip to Paris for me",
    "destination": "paris",
    "budget_usd": 2000,
    "num_days": 4,
    "travel_style": "mid-range",
    "interests": ["landmark", "museum", "activity"],
    "messages": [],
    "research_data": {},
    "budget_analysis": {},
    "itinerary": {},
    "next_agent": "researcher",
    "iteration_count": 0
}

print("🚀 Starting trip planning workflow...\n")
print("="*60)
print("USER REQUEST")
print("="*60)
print(f"Destination: {user_input['destination'].title()}")
print(f"Duration: {user_input['num_days']} days")
print(f"Budget: ${user_input['budget_usd']} USD")
print(f"Travel Style: {user_input['travel_style'].title()}")
print(f"Interests: {', '.join(user_input['interests'])}")
print("="*60)

# Run the workflow
result = app.invoke(user_input)

print("\n" + "="*60)
print("✅ WORKFLOW COMPLETE")
print("="*60)

## 📄 Display Final Itinerary

In [ ]:
# Display the final itinerary
print("\n" + "="*60)
print("🗺️  FINAL ITINERARY")
print("="*60)
print()

if result.get('itinerary'):
    itinerary = result['itinerary']
    print(f"📍 Destination: {itinerary['destination'].title()}")
    print(f"📅 Duration: {itinerary['duration_days']} days")
    print(f"💵 Budget: ${itinerary['budget_usd']} USD")
    print(f"✨ Style: {itinerary['travel_style'].title()}")
    print(f"🕐 Created: {itinerary['created_at']}")
    print("\n" + "-"*60)
    print(itinerary['itinerary_text'])
    print("-"*60)
else:
    print("❌ No itinerary generated")

# Save itinerary to file
if result.get('itinerary'):
    with open('trip_itinerary.json', 'w') as f:
        json.dump(result['itinerary'], f, indent=2)
    print("\n💾 Itinerary saved to: trip_itinerary.json")

## 🔍 View Agent Reasoning

Let's examine what each agent discovered and why they made their choices.

In [ ]:
print("\n" + "="*60)
print("🔍 AGENT REASONING & DECISIONS")
print("="*60)

print("\n📚 RESEARCHER AGENT FINDINGS:")
print("-"*60)
if result.get('research_data'):
    print(json.dumps(result['research_data'], indent=2))
else:
    print("No research data available")

print("\n💰 BUDGET AGENT ANALYSIS:")
print("-"*60)
if result.get('budget_analysis'):
    print(json.dumps(result['budget_analysis'], indent=2))
else:
    print("No budget analysis available")

print("\n💬 AGENT MESSAGES:")
print("-"*60)
for i, msg in enumerate(result.get('messages', []), 1):
    print(f"\nMessage {i}:")
    print(msg.content[:500] + "..." if len(msg.content) > 500 else msg.content)

## 🎯 Try Different Scenarios

Test the system with different destinations and preferences!

In [ ]:
# Example 2: Budget trip to Bali
bali_request = {
    "user_request": "Plan a budget-friendly trip to Bali",
    "destination": "bali",
    "budget_usd": 800,
    "num_days": 5,
    "travel_style": "budget",
    "interests": ["activity", "landmark"],
    "messages": [],
    "research_data": {},
    "budget_analysis": {},
    "itinerary": {},
    "next_agent": "researcher",
    "iteration_count": 0
}

print("🌴 Planning Bali trip...\n")
bali_result = app.invoke(bali_request)

print("\n" + "="*60)
print("🗺️  BALI ITINERARY")
print("="*60)
if bali_result.get('itinerary'):
    print(bali_result['itinerary']['itinerary_text'])

In [ ]:
# Example 3: Luxury trip to Tokyo
tokyo_request = {
    "user_request": "Plan a luxury trip to Tokyo",
    "destination": "tokyo",
    "budget_usd": 5000,
    "num_days": 6,
    "travel_style": "luxury",
    "interests": ["museum", "activity"],
    "messages": [],
    "research_data": {},
    "budget_analysis": {},
    "itinerary": {},
    "next_agent": "researcher",
    "iteration_count": 0
}

print("🗼 Planning Tokyo trip...\n")
tokyo_result = app.invoke(tokyo_request)

print("\n" + "="*60)
print("🗺️  TOKYO ITINERARY")
print("="*60)
if tokyo_result.get('itinerary'):
    print(tokyo_result['itinerary']['itinerary_text'])

## 📊 Visualize the Agent Workflow

Let's visualize how our agents collaborate!

In [ ]:
# Display the graph structure
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print("Graph visualization not available")
    print("\nWorkflow Structure:")
    print("""    
    ┌─────────────┐
    │   START     │
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐
    │ RESEARCHER  │ ← Uses: search_destination_catalog,
    │   AGENT     │         get_activity_recommendations
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐
    │   BUDGET    │ ← Uses: calculate_daily_budget,
    │   AGENT     │         calculate_trip_duration
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐
    │  PLANNER    │ ← Synthesizes all data
    │   AGENT     │   Creates final itinerary
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐
    │     END     │
    └─────────────┘
    """)

## 🎓 Summary

### What We Built:

1. **Multi-Agent System** with 3 specialized agents:
   - 🔍 **Researcher Agent**: Gathers destination information
   - 💰 **Budget Agent**: Analyzes costs and validates budget
   - 📋 **Planner Agent**: Creates detailed itineraries

2. **Custom Tools** (4 total):
   - `search_destination_catalog`: Query local destination database
   - `calculate_daily_budget`: Estimate daily costs
   - `calculate_trip_duration`: Optimize time allocation
   - `get_activity_recommendations`: Personalized suggestions

3. **Orchestration Flow**:
   - LangGraph manages agent coordination
   - Sequential workflow with shared state
   - Each agent uses tools and passes data forward

4. **Key Features**:
   - ✅ Understands traveler preferences (budget, dates, interests, style)
   - ✅ Uses multiple tools for information gathering
   - ✅ Creates day-by-day itineraries with timings and costs
   - ✅ Explains reasoning behind choices
   - ✅ Validates against budget constraints

### Technologies Used:
- **LangGraph**: Agent orchestration
- **Gemini 2.5 Flash**: LLM for agent reasoning
- **LangChain**: Tool integration and message handling
- **Python**: Core implementation

### Next Steps:
- Add more destinations to the catalog
- Integrate real-time APIs (weather, flights, hotels)
- Add user feedback loop for itinerary refinement
- Implement memory for multi-turn conversations
- Add visualization of itinerary on maps